In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

## 1. Необходимо вывести суммарное количество времени, которое сотрудник отсутствовал на рабочем месте за исключением обеденных перерывов, а также среднее время обеда. 

In [2]:
df_1 = pd.DataFrame({'name':['D', 'D', 'D', 'D', 'D', 'D', 'A', 'A', 'A', 'A', 'A', 'A'],
                     'Day_number':[1,1,1,2,2,2,1,1,1,1,2,2],
                     'Min':[15,20,63,55,20,15,0,60,5,10,20,45],
                     'Dinner_flag':['Null', 'Null', 'Y', 'Y', 'Null', 'Null', 'Null', 'Y', 'Null', 'Null', 'Null', 'Y']   
                    })
df_1

,name,Day_number,Min,Dinner_flag
0,D,1,15,Null
1,D,1,20,Null
2,D,1,63,Y
3,D,2,55,Y
4,D,2,20,Null
5,D,2,15,Null
6,A,1,0,Null
7,A,1,60,Y
8,A,1,5,Null
9,A,1,10,Null


# SQL CODE BELOW

In [3]:
#1_2_1 Необходимо вывести суммарное количество времени, которое сотрудник отсутствовал на рабочем месте за исключением обеденных перерывов

task_1_1 =  """SELECT name, Day_number, SUM (CASE WHEN Dinner_flag = 'Null' THEN min ELSE 0 END) AS total_off_time
               FROM df_1
               GROUP BY name, Day_number
            """

#1_2_1 Cреднее время обеда

task_1_2_1 = """SELECT name, AVG (CASE WHEN Dinner_flag = 'Y' THEN Min ELSE 0 END) AS avg_dinner_time
            FROM df_1
            WHERE Dinner_flag = 'Y'
            GROUP BY name
             """
#1_2_2 Cреднее время обеда

task_1_2_1 = """WITH cte AS
                (
                SELECT name, CASE WHEN Dinner_flag = 'Y' THEN Min ELSE 0 END AS avg_dinner_time
                FROM df_1
                WHERE Dinner_flag = 'Y'
                )
          SELECT name, AVG (avg_dinner_time) AS avg_dinner_time
          FROM cte
          GROUP BY name
             """
print('Total off time:')
print (sqldf(task_1_1))
print('---------------------------------')
print('Average dinner time:')
print('----------- Method 1 ------------')
print (sqldf(task_1_2_1))
print('----------- Method 2 ------------')
print (sqldf(task_1_2_1))

Total off time:
  name  Day_number  total_off_time
0    A           1              15
1    A           2              20
2    D           1              35
3    D           2              35
---------------------------------
Average dinner time:
----------- Method 1 ------------
  name  avg_dinner_time
0    A             52.5
1    D             59.0
----------- Method 2 ------------
  name  avg_dinner_time
0    A             52.5
1    D             59.0


## 2. Выведите строки с минимальной датой транзакции для каждой карты, используя оконные функции.

In [4]:
TRANS_DT = ['23.01.2017', '30.03.2017', '30.03.2017', '02.04.2017', '22.01.2017', '01.02.2017', '05.02.2017', '08.02.2017', '27.01.2017', '16.09.2017', '16.09.2017', '03.10.2017', '23.12.2017', '26.12.2017', '27.12.2017', '14.01.2017']

TRANS_DT_1 = []
for i in range(len(TRANS_DT)):
    TRANS_DT_1.append(TRANS_DT[i].replace('.', '-'))

df_2 = pd.DataFrame({'TRANS_DT':TRANS_DT_1,
                     'CARD_NUMBER':[516100009669, 516100009669, 516100009669, 516100009669, 516100004073, 516100004073, 516100004073, 516100004073, 531300007485, 520900005021, 520900005021, 520900005021, 520900003100, 520900003100, 520900003100, 516100003000],
                     'TRANS_SUM':[92, 195.74, 692.94, 110,520, 700, 957.25, 700, 159.98, 'Null', 50, 'Null', 660.5, 508.15, 266.5, 1598],
                     'MCC':[5411, 4900, 5399, 4812, 5411, 7997, 5541, 7997, 5921, 6011, 5812, 6011, 5411, 5411, 5411, 5691]   
                    })
df_2['TRANS_DT'] = pd.to_datetime(df_2['TRANS_DT'])
df_2['TRANS_DT'] = df_2['TRANS_DT'].dt.date
df_2

,TRANS_DT,CARD_NUMBER,TRANS_SUM,MCC
0,2017-01-23,516100009669,92,5411
1,2017-03-30,516100009669,195.74,4900
2,2017-03-30,516100009669,692.94,5399
3,2017-02-04,516100009669,110,4812
4,2017-01-22,516100004073,520,5411
5,2017-01-02,516100004073,700,7997
6,2017-05-02,516100004073,957.25,5541
7,2017-08-02,516100004073,700,7997
8,2017-01-27,531300007485,159.98,5921
9,2017-09-16,520900005021,Null,6011


# SQL CODE BELOW

In [5]:
task_2 = """SELECT DISTINCT CARD_NUMBER, MIN(TRANS_DT) OVER (PARTITION BY CARD_NUMBER) AS min_transaction_date
            FROM df_2
            ORDER BY CARD_NUMBER ASC;
         """
task_2_1 = """SELECT DISTINCT CARD_NUMBER, MIN(TRANS_DT) AS min_transaction_date
            FROM df_2
            GROUP BY CARD_NUMBER
            ORDER BY CARD_NUMBER ASC;
         """

print('Minimum transaction date:')
print('----------- Method 1 ------------')
print(sqldf(task_2))
print('----------- Method 2 ------------')
print(sqldf(task_2_1))

Minimum transaction date:
----------- Method 1 ------------
    CARD_NUMBER min_transaction_date
0  516100003000           2017-01-14
1  516100004073           2017-01-02
2  516100009669           2017-01-23
3  520900003100           2017-12-23
4  520900005021           2017-03-10
5  531300007485           2017-01-27
----------- Method 2 ------------
    CARD_NUMBER min_transaction_date
0  516100003000           2017-01-14
1  516100004073           2017-01-02
2  516100009669           2017-01-23
3  520900003100           2017-12-23
4  520900005021           2017-03-10
5  531300007485           2017-01-27


## 3. Клиенты могут брать в долг, копить, либо одновременно и копить и иметь кредиты: Баланс клиента = сумма депозитов клиента – сумма кредитов клиента 
## Необходимо узнать: А) Какая доля клиентов имеет положительный баланс? Б) Вывести распределение положительных балансов по бакетам: 0-100 тыс., 100-300 тыс., более 300 тыс.

In [6]:
df_3 = pd.DataFrame({'id_credit':[458000263, 468010954, 773002215],
                     'id_client':[100001, 100001, 100002],
                     'type':['cash', 'credit card', 'credit card'],
                     'balance':[250000, 70000, 50000]   
                    }, index=[0,1,2])

df_3_1 = pd.DataFrame({'id_deposit':[640025673, 330000456, 468030123],
                     'id_client':[100006, 100007, 100002],
                     'type':['deposit', 'invest', 'debit card'],
                     'balance':[600000, 130000, 16000]   
                    }, index=[4,3,2])

df_3 = df_3.rename(columns={'id_client':'id_client_credit', 'type':'type_credit', 'balance':'balance_credit'})
df_3_1 = df_3_1.rename(columns={'id_client':'id_client_deposit', 'type':'type_deposit', 'balance':'balance_deposit'})
df_3_full_join = pd.concat([df_3_1, df_3], join='outer', axis=1)
df_3_full_join = df_3_full_join.fillna(0)
df_3_full_join['id_credit'] = df_3_full_join['id_credit'].astype('int')
df_3_full_join['id_client_credit'] = df_3_full_join['id_client_credit'].astype('int')
df_3_full_join['id_deposit'] = df_3_full_join['id_deposit'].astype('int')
df_3_full_join['id_client_deposit'] = df_3_full_join['id_client_deposit'].astype('int')
df_3_full_join = df_3_full_join.fillna('Null')
df_3_full_join

,id_deposit,id_client_deposit,type_deposit,balance_deposit,id_credit,id_client_credit,type_credit,balance_credit
0,0,0,0,0.0,458000263,100001,cash,250000.0
1,0,0,0,0.0,468010954,100001,credit card,70000.0
2,468030123,100002,debit card,16000.0,773002215,100002,credit card,50000.0
3,330000456,100007,invest,130000.0,0,0,0,0.0
4,640025673,100006,deposit,600000.0,0,0,0,0.0


# SQL CODE BELOW

In [7]:
# A) Какая доля клиентов имеет положительный баланс?

task_3_share = """SELECT over_zero:NUMERIC / total_clients::NUMERIC *100 AS share_of_over_zero 
             FROM (  SELECT *, 
                    (balance_deposit-balance_credit) AS total_balance, 
                     COUNT (*) OVER () AS total_clients, 
                     SUM(CASE WHEN total_balance > 0 THEN 1 ELSE 0 END) AS over_zero,
                     SUM(CASE WHEN total_balance < 0 THEN 1 ELSE 0 END) AS less_zero
            FROM df_3 a
            FULL JOIN df_3_1 b
            ) a
            ;
         """
print('share of client that have positive balance: 40%')

# Б) Вывести распределение положительных балансов по бакетам: 0-100 тыс., 100-300 тыс., более 300 тыс.

task_3_bucket_1 = """SELECT *, CASE WHEN (balance_deposit-balance_credit)>0 AND (balance_deposit-balance_credit) BETWEEN 0 AND 100000 THEN '0-100k'
                                  WHEN (balance_deposit-balance_credit)>0 AND (balance_deposit-balance_credit) BETWEEN 100000 AND 300000 THEN '100-300k'
                                  WHEN (balance_deposit-balance_credit)>0 AND (balance_deposit-balance_credit) > 300000 THEN 'Over 300k'
                                  ELSE 'Negative balance'
                                  END segments
                   FROM df_3_full_join
            ; 
         """
task_3_bucket_2 = """SELECT id_client_deposit, segments
                     FROM (
                             SELECT *, (balance_deposit-balance_credit) AS total_balance,
                             CASE WHEN (balance_deposit-balance_credit)>0 AND (balance_deposit-balance_credit) BETWEEN 0 AND 100000 THEN '0-100k'
                                  WHEN (balance_deposit-balance_credit)>0 AND (balance_deposit-balance_credit) BETWEEN 100000 AND 300000 THEN '100-300k'
                                  WHEN (balance_deposit-balance_credit)>0 AND (balance_deposit-balance_credit) > 300000 THEN 'Over 300k'
                                  END segments
                     FROM df_3_full_join
                     ) a
                     WHERE total_balance >=0
            ; 
         """

print('------------------------------------------------------------------')
print('Positive balance by bucket:')
sqldf(task_3_bucket_2)

share of client that have positive balance: 40%
------------------------------------------------------------------
Positive balance by bucket:


,id_client_deposit,segments
0,100007,100-300k
1,100006,Over 300k


## 4. Отток по картам составляет 4%* в месяц. Каков средний срок жизни карты?
### * 4% от активных карт на начало месяца закрываются в течение месяца

In [8]:
# Attempt 1

churn_rate = 0.04 # 4% 
final_formula = int(1 / churn_rate) # 100% / Churn rate = Average Customer Lifetime

print(f'Method 1: ACL = {final_formula} months')

# Attempt 2

qty_of_card = 1000 # for example we've got 1000 card
share_of_churn = qty_of_card * churn_rate
absolut_churn = 0
total_absolut_churn = []
counts = 0

while absolut_churn < qty_of_card:
    absolut_churn += share_of_churn
    counts += 1
    
print('------------------------------------------')    
print(f'Method 2: ACL = {int(counts)} months')  

Method 1: ACL = 25 months
------------------------------------------
Method 2: ACL = 25 months
